In [ ]:
%%!pip install torch torchvision

     -------------------------------------- 241.2/241.2 MB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 6.3/6.3 MB 3.9 MB/s eta 0:00:00
  Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
     -------------------------------------- 201.0/201.0 KB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 4.8 MB/s eta 0:00:00
     -------------------------------------- 536.2/536.2 KB 3.8 MB/s eta 0:00:00
  Using cached markupsafe-3.0.3-cp39-cp39-win_amd64.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\nabiya\Downloads\Gestura-Model-main\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(
    root="../datasets/asl_alphabet",
    transform=transform
)

print("Total images:", len(dataset))
print("Classes:", dataset.classes)


Total images: 2427
Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


In [3]:
import torch
import torch.nn as nn

class StaticCNN(nn.Module):
    def __init__(self, num_classes=24):  # no J, no Z
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(32 * 14 * 14, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


model = StaticCNN()
print(model)


StaticCNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=6272, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=24, bias=True)
  )
)


In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Loss and optimizer ready")


Loss and optimizer ready


In [5]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=32, shuffle=True)

epochs = 5

for epoch in range(epochs):
    running_loss = 0.0

    for images, labels in loader:
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {running_loss:.4f}")


Epoch [1/5] - Loss: 241.7287
Epoch [2/5] - Loss: 239.0671
Epoch [3/5] - Loss: 224.1272
Epoch [4/5] - Loss: 174.9743
Epoch [5/5] - Loss: 121.5500


In [6]:
import os
os.makedirs("../models", exist_ok=True)

torch.save(model.state_dict(), "../models/static_cnn.pth")
print("static_cnn.pth saved")


static_cnn.pth saved
